In [52]:
from simulators.epidemiology import make_system, update_func, run_simulation
from models.recurrent_nn import RNN
from data.epidemiology import EpidemiologyDataset
from torch.utils.data import DataLoader
import pytorch_lightning as pl
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
ds = EpidemiologyDataset(1/5, 1/8, 25)
dataloader = DataLoader(ds, num_workers=4)

In [3]:
for x, y in dataloader:
    break

In [27]:
s = x[0, :, 0]
i = y[0, :]
r = x[0, :, 1]
beta = 1/5
gamma = 1/8

In [28]:
print("s:", s[0])
print("i:", i[0])
print("r:", r[0])

s: tensor(0.8967)
i: tensor(0.0924)
r: tensor(0.0109)


**KM Equations**:
\begin{align*}
s_{i} &= s_{i-1} - \beta\,s_{i-1}\,i_{i-1}\\
i_{i} &= i_{i-1} + i_{i-1}(\beta s_{i-1} - \gamma)\\
r_{i} &= r_{i-1} + \gamma\,i_{i-1}
\end{align*}

We can expand the KM equations to predict further out into the future (e.g. for two steps):
\begin{align*}
s_i &= s_{i-2} - \beta(s_{i-1}\cdot i_{i-1} + s_{i-2}\cdot i_{i-2})\\
i_i &= i_{i-2} + [i_{i-1}(\beta\,s_{i-1}-\gamma) + i_{i-2}(\beta\,s_{i-2}-\gamma)]\\
r_i &= r_{i-2} + \gamma(i_{i-1} + i_{i-2})
\end{align*}

In [50]:
print("s[2]:", (s[0]-(beta*(s[1]*i[1]+s[0]*i[0])).item()).item())
print("i[2]:", (i[0]+((beta*s[1]*i[1]-gamma*i[1])+(beta*s[0]*i[0]-gamma*i[0]))).item())
print("r[2]:", (r[0]+gamma*(i[1]+i[0])).item())

s[2]: 0.8629523515701294
i[2]: 0.10239651054143906
r[2]: 0.03465118631720543


In [51]:
print("True s[2]:", s[2].item())
print("True i[2]:", i[2].item())
print("True r[2]:", r[2].item())

True s[2]: 0.8629522919654846
True i[2]: 0.10239650309085846
True r[2]: 0.03465118631720543
